In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:

def OneMax(P):
    return np.sum(P, axis=1)
    
def DeceptiveOneMax(P):
    length = len(P[0])
    P = np.sum(P,axis=1)
    for index,x in enumerate(P):
        if x == 0:
            P[index] = length + 1
    return P 

def helper(x):
    length = len(x)
    x = np.sum(x)
    if x == 0:
        return length +1
    else:
        return x

def KDeceptiveOneMax(P, k=6):
    result = []
    for x in P:
        res = np.sum([helper(x[(i*k):((i+1)*k)]) for i in range(int(np.floor(len(x)/k)))])
        result.append(res)
    return result




[12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0]

In [ ]:
def Random_Population(p, N):
    length = len(p) 
    P = np.zeros((N, length))
    randoms = np.random.random_sample((N,length))
    for index, x in enumerate(randoms):
        p_row = np.where(x< p,1,0)
        P[index] = p_row
    return P    

def entropy(v):
    uq = np.unique(v)
    size = len(v)
    entropy = np.sum([(np.sum(v == x) / size) * np.log(np.sum(v==x)/size) for x in uq])
    return entropy

def best_entropy(P):
    s = P.shape[1]
    P = P.T
    mini_entropy = np.Inf
    I_n = 0
    for i in range(s):
        if entropy(P[i]) < mini_entropy:
            I_n = i
            mini_entropy = entropy(P[i])
        
    return I_n



def Cond_Entropy(iK, i_pk, P):
    size = P.shape[0]
    P = P.T
    uq = np.unique(P[iK])
    counts = {}
    for x in uq:
        counts[x] = len(P[i_pk][P[iK]==x]) / size

    cond_entropy = [counts[x] * entropy(P[i_pk][P[iK]==x]) for x in uq]
    cond_entropy = np.sum(cond_entropy)

    return cond_entropy


def Probability_estimation(p, P):
    i_n = best_entropy(P)
    length = P.shape[1]

    possible_i = np.array(np.arange(length))
    index = np.where(possible_i == i_n) 
    possible_i = np.delete(possible_i, index) ## Possible indexes for next best column

    Is = []
    Is.append(i_n) # List of indexes with columns

    i_k_1 = i_n # We start with i_n

    for i in reversed(range(length-1)):
        mini = np.Inf
        index = 0
        for x in possible_i:
            if mini > Cond_Entropy(i_k_1, x, P):
                index = x
                mini = Cond_Entropy(i_k_1, x, P)

        Is.append(index)
        i_k_1 = index
        index = np.where(possible_i == index)
        possible_i = np.delete(possible_i, index) # Deleting index that was just chosen and setting new i_k+1

    p = p_estimation(p, P, Is)

    return p

def p_estimation(p, P, Is):
    print(Is)
    Is = np.array(Is)
    length = P.shape[0]
    P = P.T
    for index,x in enumerate(Is):
        if index==0:
            
            p[x] = np.sum(P[x]) / length

        else:
            length = np.sum(P[Is[index-1]] == 1)
            p[x] = np.sum(P[x][P[Is[index-1]]== 1]) / length
    print(p)
    print('end')
    return p






In [ ]:

def MIMIC(population_size, chromosome_length, objective_func, max_iter=200, Nth_percentile = 25):
    p = np.ones(chromosome_length)/2
    
    population = Random_Population(p, population_size)
    objective_values = objective_func(population)
    
    theta  = np.mean(objective_values)
    
    indexs =  np.where(objective_values >= theta)
    population = population[indexs]
    p = Probability_estimation(p, population)

    for i in range(max_iter):
        population = Random_Population(p, population_size)
        objective_values = objective_func(population) # Getting new population and calculating objectives values for that population

        theta  = np.percentile(objective_values, Nth_percentile) # Setting new theta
    
        indexs =  np.where(objective_values>=theta)
        population = population[indexs] # Getting only best Nth_percentile from population
        
        p = Probability_estimation(p, population) # Estimating new probabilities

        if objective_func== OneMax:
            if (population[0] == np.ones(chromosome_length)).all():
                print('MIMIC')
                print(i)
                break
        if objective_func== DeceptiveOneMax:
            if (population[0] == np.ones(chromosome_length)).all():
                print('MIMIC')
                print(i)
                break
        if objective_func == KDeceptiveOneMax:
            if (population[0] == np.ones(chromosome_length)).all():
                print('MIMIC')
                print(i)
                break


    return theta, population,objective_values, p


theta, population, objective_values, probabilities = MIMIC(125,50,OneMax,200)


[13, 31, 11, 15, 33, 27, 17, 23, 22, 34, 28, 20, 45, 39, 9, 24, 41, 44, 26, 40, 5, 36, 43, 47, 42, 35, 46, 3, 12, 32, 29, 14, 30, 10, 4, 38, 19, 6, 0, 48, 25, 18, 2, 7, 8, 16, 37, 1, 21, 49]
[0.61111111 0.65       0.64864865 0.55882353 0.62857143 0.55882353
 0.55555556 0.66666667 0.61538462 0.54545455 0.6        0.55172414
 0.58823529 0.50819672 0.61538462 0.53125    0.64102564 0.51724138
 0.54054054 0.6        0.48387097 0.625      0.48275862 0.46875
 0.54545455 0.66666667 0.53571429 0.48387097 0.51612903 0.4
 0.6        0.48387097 0.57142857 0.53125    0.5        0.52941176
 0.55882353 0.65       0.52777778 0.55172414 0.54545455 0.51515152
 0.52941176 0.55882353 0.45454545 0.5        0.52941176 0.55882353
 0.37837838 0.675     ]
end
[11, 35, 41, 22, 15, 9, 44, 5, 10, 31, 33, 27, 20, 47, 45, 6, 42, 46, 24, 18, 26, 16, 34, 23, 40, 36, 13, 17, 14, 38, 1, 8, 3, 37, 2, 29, 39, 19, 30, 28, 43, 48, 0, 21, 32, 12, 7, 4, 49, 25]
[0.66666667 0.6        0.63934426 0.65       0.76470588 0.530612

In [ ]:
theta, population, objective_values, probabilities = MIMIC(125,50,KDeceptiveOneMax,500)

MIMIC
39


In [ ]:
theta, population, objective_values, probabilities = MIMIC(125,50,DeceptiveOneMax,200)


[1, 33, 18, 24, 39, 2, 46, 4, 26, 19, 42, 16, 21, 6, 12, 23, 35, 0, 8, 22, 27, 38, 32, 11, 14, 48, 47, 43, 49, 36, 20, 30, 3, 9, 40, 10, 17, 34, 45, 28, 5, 31, 41, 37, 44, 7, 15, 13, 29, 25]
[0.56666667 0.50877193 0.51724138 0.60606061 0.46428571 0.6
 0.48148148 0.71052632 0.5625     0.60606061 0.58823529 0.5625
 0.48148148 0.69230769 0.5483871  0.71052632 0.48275862 0.55882353
 0.51724138 0.53571429 0.59375    0.48148148 0.5483871  0.51724138
 0.51724138 0.69230769 0.48148148 0.51612903 0.62857143 0.69230769
 0.54545455 0.65714286 0.55555556 0.51724138 0.61764706 0.5
 0.5625     0.61111111 0.4516129  0.51724138 0.61764706 0.62857143
 0.5        0.40625    0.66666667 0.58823529 0.5        0.53846154
 0.4375     0.52      ]
end
[16, 21, 30, 2, 1, 35, 47, 6, 19, 45, 26, 42, 22, 12, 11, 43, 46, 48, 27, 14, 0, 39, 33, 4, 10, 38, 5, 23, 17, 18, 8, 32, 24, 37, 20, 34, 49, 7, 28, 36, 3, 41, 9, 44, 29, 40, 31, 15, 25, 13]
[0.5        0.49019608 0.52083333 0.68253968 0.53571429 0.625
 0.5      